**Context Menu TODO**

* Convert the old context menus to the new system.
* Make the new context menu behave more like system menus.
    * Position menu so it never goes offscreen.

In [1]:
import xml.etree.ElementTree as xml

import numpy

import toyplot.html
import toyplot.mark

import logging
logging.basicConfig(level=logging.DEBUG)

toyplot.log.setLevel(logging.DEBUG)

In [2]:
# Custom marks must derive from toyplot.mark.Mark
class Circle(toyplot.mark.Mark):
    def __init__(self, x, y, radius):
        super(Circle, self).__init__()
        self._coordinates = {
            "x": x,
            "y": y,
        }
        self._radius = radius
        
    # Marks should return their domain (their minimum and maximum values along a given axis).
    def domain(self, axis):
        return numpy.array(self._coordinates[axis]), numpy.array(self._coordinates[axis])
    
    # Marks should return their extents, which are a combination of domain coordinates and
    # range extents relative to those coordinates.  This is so the domain and range of an axis can
    # be adjusted to account for the size of the mark (in range coordinates)
    def extents(self, axes):
        coordinates = tuple([numpy.array([self._coordinates[axis]]) for axis in axes])
        extents = (
            numpy.array([-self._radius]),
            numpy.array([self._radius]),
            numpy.array([-self._radius]),
            numpy.array([self._radius]),
            )
        return coordinates, extents

# Custom marks must define a _render() function and register it using dispatch() so it can be
# called at render time.  Note that _render() is registered for a given combination of coordinate
# system and mark.  This allows marks to adapt their visual representation to the coordinate
# system (for example, a scatterplot mark would be rendered using lines if it was part of a
# hypothetical parallel coordinate system).
@toyplot.html.dispatch(toyplot.coordinates.Cartesian, Circle, toyplot.html.RenderContext)
def _render(axes, mark, context):
    x = axes.project("x", mark._coordinates["x"])
    y = axes.project("y", mark._coordinates["y"])
    xml.SubElement(
        context.parent,
        "circle",
        id=context.get_id(mark),
        cx=repr(x),
        cy=repr(y),
        r=str(mark._radius),
        stroke="black",
        fill="lightgray",
        )
    
    circle_ml = "<circle x=%s y=%s radius=%s></circle>" % (mark._coordinates["x"], mark._coordinates["y"], mark._radius)
    
    context.require(["toyplot/menu/context", "toyplot/file"], """function(contextmenu, file, mark_id, document) {
        function show_item(e)
        {
            return e.target.id == mark_id;
        }
        
        function choose_item()
        {
            file.save("text/xml+cml", "utf-8", document, "test.cml");
        }
        contextmenu.add_item("Save circle as CSV", show_item, choose_item);
        contextmenu.add_item("Save circle as CircleML", show_item, choose_item);
        contextmenu.add_item("Save docoument as PNG", show_item, choose_item);
        }""", arguments=[context.get_id(mark), circle_ml])

In [6]:
canvas = toyplot.Canvas()
axes = canvas.cartesian()
axes.scatterplot(numpy.linspace(2, 3, 1000), numpy.random.normal(2.5, 0.1, size=1000))
axes.add_mark(Circle(2, 3, 20))
axes.add_mark(Circle(3, 2, 50));

<circle cx="479.79821927190346" cy="292.63604345138049" r